# Managing the grid from python

As the grid is completely bi-directional, you can act on the grid from python. Here are some interesting features that have been implemented.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import pandas as pd
import numpy as np
import urllib.request as ur
from copy import deepcopy as copy
from ipyaggrid import Grid


In [3]:
url = 'https://raw.githubusercontent.com/bahamas10/css-color-names/master/css-color-names.json'
with ur.urlopen(url) as res:
    cnames = json.loads(res.read().decode('utf-8'))

colors = []
for k in cnames.keys():
    colors.append({'color':k, 'value':cnames[k]})


In [4]:
colors_ref = colors[:]

## Exporting data via python

In [5]:
css_rules="""
.color-box{
  float: left;
  width: 10px;
  height: 10px;
  margin: 5px;
  border: 1px solid rgba(0, 0, 0, .2);
}
"""

columnDefs = [
    {'headerName': 'Color', 'field':'color', 
     'pinned': True, 'editable': True},
    {'headerName': 'Code', 'field':'value', 'editable': False, 'cellRenderer': """
        function(params){
            return `<div><div style="background-color:${params.value}" class='color-box'></div><span>${params.value}</span></div>`
      }"""}
]

gridOptions = {'columnDefs':columnDefs,
              'enableFilter':'true',
              'enableSorting':'true',
              'rowSelection':'multiple',
              }

color_grid = Grid(width=400,
                 height=250,
                 css_rules=css_rules,
                 grid_data=colors,
                 grid_options=gridOptions,
                 sync_on_edit=True,
                 sync_grid=True, #default
                 )
display(color_grid)


Grid(columns_fit='size_to_fit', compress_data=True, css_rules_down=['.color-box{float: left;width: 10px;height…

You can use the `get_grid`, `get_selected_rows`, `get_selected_columns` (available only in rangeSelection mode) to get the data out of the grid even when deactivating the export mode.

In [6]:
color_grid.get_grid()
color_grid.get_selected_rows()


In [10]:
color_grid.grid_data_out.get('grid')

,color,value
Index,,
0,aliceblue,#f0f8ff
1,antiquewhite,#faebd7
2,aqua,#00ffff
3,aquamarine,#7fffd4
4,azure,#f0ffff
...,...,...
143,wheat,#f5deb3
144,white,#ffffff
145,whitesmoke,#f5f5f5


If you set `sync_grid=True` (default) then the grid data is automatically synced with `grid_data_out['grid']` as a dataframe.

In [11]:
color_grid.grid_data_out.get('grid')

,color,value
Index,,
0,aliceblue,#f0f8ff
1,antiquewhite,#faebd7
2,aqua,#00ffff
3,aquamarine,#7fffd4
4,azure,#f0ffff
...,...,...
143,wheat,#f5deb3
144,white,#ffffff
145,whitesmoke,#f5f5f5


## Updating Data

In [12]:
gridOptions = {'columnDefs':columnDefs,
               'enableFilter':'true',
               'enableColumnResize':'true',
               'enableSorting':'true',
               'rowSelection':'multiple',
              }

color_grid2 = Grid(width=500,
                 height=250,
                 css_rules=css_rules,
                 quick_filter=True,
                 show_toggle_edit=True,
                 grid_data=colors_ref,
                 grid_options=gridOptions)
color_grid2

Grid(columns_fit='size_to_fit', compress_data=True, css_rules_down=['.color-box{float: left;width: 10px;height…

In [13]:
colors = colors_ref[:]
colors.append({'color':'jupyterorange', 'value':'#f37626'})

In [14]:
color_grid2.update_grid_data(copy(colors)) # New data set corresponding to the good columns

In [15]:
color_grid2.delete_selected_rows()

In [16]:
color_grid2.grid_data_out['grid']

,color,value
Index,,
0,aliceblue,#f0f8ff
1,antiquewhite,#faebd7
2,aqua,#00ffff
3,aquamarine,#7fffd4
4,azure,#f0ffff
...,...,...
142,violet,#ee82ee
143,wheat,#f5deb3
144,white,#ffffff
